In [1]:
import pandas as pd
#import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import keras

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
path = './data/'
ll_data = 'leaf_level/group/'
csv = '.csv'

In [ ]:
# Load the data into a Pandas DataFrame
mkt_data = pd.read_csv(path+'power_market_MA'+csv, parse_dates = ["date"])
mkt_data.sort_values("date", inplace=True)
mkt_data.drop('Unnamed: 0', axis=1, inplace=True)
supp_data = pd.read_csv(path+ll_data+'Austru_85055_20000_Musharna'+csv, parse_dates = ["date"])
supp_data.sort_values("date", inplace=True)

In [ ]:
supp_data = supp_data.groupby('date').agg({'price_year_eur':'mean'})

In [ ]:
supp_data = supp_data.merge(mkt_data, how='inner', on='date')

In [ ]:
new_mkt = supp_data.copy()
new_mkt.drop(columns = ['price_year_eur'], inplace = True)
new_mkt.set_index('date', inplace=True)

In [ ]:
X= new_mkt

In [ ]:
X

In [ ]:
 # Scaling independent variable
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y = supp_data['price_year_eur']

In [ ]:
# Using PCA to find the most relevant price instruments
pca = PCA(0.99)
X_pca = pca.fit_transform(X_scaled)
num_ins = pca.n_components_

In [ ]:
pca.n_components_

In [ ]:
# Split the data into training and test sets
X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

In [ ]:
# Normalize the data
X_train_pca = (X_train_pca - X_train_pca.mean()) / X_train_pca.std()
X_test_pca = (X_test_pca - X_test_pca.mean()) / X_test_pca.std()

In [ ]:
# Build the model using the TensorFlow Keras API
model = keras.Sequential()
model.add(keras.layers.Dense(units=32, input_dim=num_ins, activation='relu'))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=32, activation='relu'))
model.add(keras.layers.Dense(units=1, activation='linear'))

In [ ]:
# Compile the model
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.001), loss='mean_squared_error')

In [ ]:
# Train the model on the training data
history = model.fit(X_train_pca, y_train, epochs=10, batch_size=32)

In [ ]:
# Use the model to make predictions on the test data
y_pred = model.predict(X_test_pca)

In [ ]:
# Calculate the mean squared error on the test data
mse = mean_squared_error(y_test, y_pred)
print('Mean squared error:', mse)

In [ ]:
model.score(y_test, y_pred)